In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import openpyxl 

In [2]:
goty_df = pd.read_excel("goty.xlsx")

In [3]:
goty_df.nunique() # Total number of unique values in each column

Name                 62
Developer            39
Publisher            28
Nominations           9
Meta-Score           16
Reviews              44
User-Score           26
Votes                62
Genre                 6
Release-Date         59
Year of the Event    11
Popularity           19
Most-Anticipated      2
Wins                  2
dtype: int64

In [4]:
goty_df.shape

(62, 14)

In [5]:
goty_df.columns

Index(['Name', 'Developer', 'Publisher', 'Nominations', 'Meta-Score',
       'Reviews', 'User-Score', 'Votes', 'Genre', 'Release-Date',
       'Year of the Event', 'Popularity', 'Most-Anticipated', 'Wins'],
      dtype='object')

In [6]:
goty_df['Release Month'] = goty_df['Release-Date'].dt.month

In [7]:
goty_df[goty_df['Release Month'] == 12] 

# They are nominating the games even in released december.

,Name,Developer,Publisher,Nominations,Meta-Score,Reviews,User-Score,Votes,Genre,Release-Date,Year of the Event,Popularity,Most-Anticipated,Wins,Release Month
27,Super Smash Bros. Ultimate,Sora. Ltd,Nintendo,3,93,99,87,5055,Fighting,2018-12-07,2019,11,0,0,12


In [8]:
goty_df['Year of the Event'] = pd.to_datetime(
    goty_df['Year of the Event'], 
    format='%Y').dt.year

In [9]:
goty_df[goty_df['Year of the Event'] != goty_df['Release-Date'].dt.year]
# That is the only game not released in the year of the event? 
# Double check the release date of the game on internet for validation

,Name,Developer,Publisher,Nominations,Meta-Score,Reviews,User-Score,Votes,Genre,Release-Date,Year of the Event,Popularity,Most-Anticipated,Wins,Release Month
27,Super Smash Bros. Ultimate,Sora. Ltd,Nintendo,3,93,99,87,5055,Fighting,2018-12-07,2019,11,0,0,12


In [10]:
goty_df.groupby([(goty_df['Genre']), (goty_df['Wins']== 1)])[
    'Meta-Score'].agg(['mean','count'])
# Avg meta scores of genres that have won or lost

mean  count
Genre            Wins                   
Action           False  87.571429      7
                 True   91.000000      1
Action-Adventure False  88.827586     29
                 True   92.666667      6
Fighting         False  93.000000      1
RPG              False  90.363636     11
                 True   92.250000      4
Simulation       False  90.000000      1
Strategy         False  89.000000      2

In [11]:
gecen_nomineeler = goty_df[
    ((goty_df['Nominations'] == goty_df['Nominations'].max()) & (goty_df['Wins'] == 1)) | 
    (goty_df['Nominations'] == 0)
]['Nominations'].unique()
nihai_veri = goty_df[goty_df['Nominations'].isin(gecen_nomineeler)]
nihai_veri

# Most nominated games

,Name,Developer,Publisher,Nominations,Meta-Score,Reviews,User-Score,Votes,Genre,Release-Date,Year of the Event,Popularity,Most-Anticipated,Wins,Release Month
32,The last of us Part 2,Naughty Dog,PlayStation,10,93,121,58,166250,Action-Adventure,2020-06-19,2020,7,0,1,6
47,God of War Ragnarok,Santa Monica,PlayStation,10,94,141,82,14896,Action-Adventure,2022-11-09,2022,2,1,0,11


In [12]:
goty_df.groupby('Wins')['Nominations'].agg(['mean'])

,mean
Wins,
0,4.980392
1,6.363636


In [22]:
goty_df.groupby(['Publisher'])['Nominations'].sum().sort_values(ascending=False)
# Total nominations per publisher

Publisher
PlayStation           87
Nintendo              38
Bethesda Softworks    20
Capcom                15
Bandai Namco          14
Square Enix           13
Electronic Arts       10
Sega                  10
Blizzard               9
Warner Bros            8
Supergiant Games       8
Rockstar Games         8
505 Games              8
Larian Studios         8
Epic Games             8
Playstation            7
Annapurna              6
Activision             5
Playdead               5
Playstack              5
Focus                  5
Xbox                   5
Matt Makes Games       4
Private Division       4
Konami                 4
Game Science           4
Ubisoft                4
PUBG Corp              2
Name: Nominations, dtype: int64

In [23]:
goty_df.groupby(['Publisher'])['Name'].count().sort_values(ascending=False)
# Total number of games per publisher in TGA

Publisher
PlayStation           12
Nintendo              10
Bethesda Softworks     4
Capcom                 4
Bandai Namco           3
Electronic Arts        3
Warner Bros            2
Blizzard               2
Square Enix            2
Sega                   2
Playdead               1
Supergiant Games       1
Rockstar Games         1
Private Division       1
Playstation            1
Playstack              1
Ubisoft                1
505 Games              1
PUBG Corp              1
Activision             1
Matt Makes Games       1
Larian Studios         1
Konami                 1
Game Science           1
Focus                  1
Epic Games             1
Annapurna              1
Xbox                   1
Name: Name, dtype: int64

In [24]:
goty_df[(goty_df['Most-Anticipated'] == 1) & (goty_df['Wins'] == 1)][
    'Name'].count() * (100 / goty_df['Name'].count())

# Hype is not a good metric for predict the winner

np.float64(6.451612903225806)

In [16]:
goty_df['log_votes'] = np.log(goty_df['Votes']+1)
goty_df['Public Score'] = goty_df['log_votes'] * goty_df['User-Score']


In [17]:
def normalize_sutun(series):
    return (series - series.min()) / (series.max() - series.min())

goty_df['critic_score_norm'] = normalize_sutun(goty_df['Meta-Score'])
goty_df['votes_norm'] = normalize_sutun(goty_df['log_votes'])

goty_df['winning_score'] = (goty_df['critic_score_norm'] * 0.90) + \
                                 (goty_df['votes_norm'] * 0.10)

In [18]:
print(goty_df[['Name','Meta-Score', 'Votes', 'winning_score','Wins']].sort_values(
    'winning_score', ascending=False).head(10))

                                         Name  Meta-Score   Votes  \
23                     Red Dead Redemption 2           97   32418   
16    The Legend of Zelda: Breath of the Wild          97   25410   
19                        Super Mario Oddysey          97   10761   
48                                 Elden Ring          96   21412   
54                          Baldur's Gate III          96   19706   
52  The Legend of Zelda: Tears of the Kingdom          96   13804   
20                                 God of War          94   29061   
47                        God of War Ragnarok          94   14896   
32                      The last of us Part 2          93  166250   
56                                  Astro Bot          94    5319   

    winning_score  Wins  
23       0.967851     0  
16       0.963061     1  
19       0.946165     0  
48       0.903444     1  
54       0.901812     1  
52       0.894812     0  
20       0.796951     1  
47       0.783809     0  
32    

In [ ]:
goty_df['Votes'].argmax()

# Index of the game that got maximum votes from public

np.int64(32)

In [ ]:
goty_df['Votes'].argmin()

# Index of the game that got minimum votes from public